In [1]:
import random
import json
from tqdm import tqdm
import numpy as np
import torch
import gc 

import konlpy
from multiprocess import Pool

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [4]:
train = load_jsonl('dataset/bfly/train.jsonl')
valid = load_jsonl('dataset/bfly/dev.jsonl')
#train = [article['article_original'] for article in train]

Loaded 10000 records from dataset/bfly/dev.jsonl


## TRAIN

In [94]:
# minimum 6 sentences
train_min = [doc for doc in train if len(doc['article_original']) >= 6]
print(f'{len(train)} --> {len(train_min)}')

260697 --> 256998


In [95]:
abs_set = [doc['abstractive'] for doc in train_min]

In [96]:
article_set = [doc['article_original'] for doc in train_min]

In [5]:
#okt = konlpy.tag.Okt()

In [28]:
# from konlpy.tag import Okt
# okt = None

# def multiprocessing_initializer():
#     global okt
#     okt = Okt()

# def get_nouns(sentence):
#     nouns = okt.nouns(sentence)
#     res = ' '.join(nouns)
#     return res

In [97]:
tot_num = 100000

In [98]:
y_set_abs = abs_set[:tot_num]
y_set_doc = article_set[:tot_num]

n_set_abs = abs_set[tot_num:]
n_set_doc = article_set[tot_num:]

In [99]:
okt = konlpy.tag.Okt()

In [100]:
import gc
abs_set = []
article_set = []
gc.collect()

22

In [153]:
# word_list = []
# for ab in tqdm(y_set_abs):
#     nouns = okt.nouns(ab)
#     word_list.append(' '.join(nouns))

In [55]:
# import torch
# torch.save(word_list, 'word_list.pt')

In [2]:
word_list = torch.load('word_list.pt')

In [3]:
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(word_list)

In [81]:
tot_num = 100000
batch_size = 1000

visited = [0]*tot_num
pairs = []
all_sims = []
idx = 0
cnt = 0

stop = False
while idx < tot_num and not stop:
    print(idx) if idx % 10000 == 0 else None
    lh, rh = tfidf_matrix[idx:idx+batch_size], tfidf_matrix
    sim = cosine_similarity(lh, rh)[:, idx:]
    sim[np.tril_indices(batch_size)] = 0.0
    
    for i, r in enumerate(sim):
        i_real = i + idx
        sim_max = np.argmax(r)
        if (visited[i_real] == 0) and (visited[sim_max+idx] == 0) and (0.05 <= r[sim_max] < 0.7):
            visited[i_real] = visited[sim_max + idx] = 1
            pairs.append((i_real, sim_max+idx))
            cnt += 2
#         if cnt > 70000:
#             stop = True
#             break

    idx += batch_size

sim = []
gc.collect()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


8

In [102]:
paired_docs = [[y_set_doc[l], y_set_doc[r]] for l, r in pairs]

In [87]:
paired_list = [i for i, v in enumerate(visited) if v == 1]
left_list = [i for i, v in enumerate(visited) if v == 0]

In [126]:
i = 0
random_docs = []
while True:
    lh, rh = left_list[i], left_list[i+1]
    random_docs.append([y_set_doc[lh], y_set_doc[rh]])
    
    i += 2
    if i <= len(left_list) - 2:
        pass
    else:
        break

In [135]:
paired_docs[0]

[['지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다.',
  '폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계, 두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다.',
  '‘연령표준화 사망률’(표준인구 10만 명당 사망자 수)은 울산·충북·부산 순으로 높게 나타났다.',
  '■작년 사망자 29만 8820명, 역대 최다',
  "24일 통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 전년 대비 4.7%(1만 3286명) 증가한 29만 8820명으로 관련 통계를 작성한 1983년 이후 가장 많았으며, 5년 연속 증가세를 보였다.",
  '통계청은 인구 구조의 고령화와 지난해 1~2월 유례가 드문 한파 등을 그 원인으로 꼽았다.',
  '지난해 조사망률(인구 10만 명당 사망자 수) 역시 582.5명으로 전년보다 4.5%(25.1명) 증가해 5년 연속 늘었다.',
  '특히 80세 이상의 사망자가 전체 사망자의 절반에 가까운 46.3%로, 10년 전보다 14.3%포인트(P)나 증가했다.',
  '■폐렴·알츠하이머병 사망률 순위 ‘껑충’',
  '사망원인별로 보면 지난해 암(악성신생물)에 의한 사망률(이하 인구 10만 명당 사망자 수)은 154.3명으로 전년보다 0.2% 증가했다.',
  '1983년 관련 통계를 집계한 이래 줄곧 암이 사망원인 1위로 집계됐다.',
  '특히 폐렴(4위→3위)과 치매의 일종인 알츠하이머병(11위→9위)에 의한 사망률 순위 상승이 두드러졌다.',
  '폐렴 사망률은 2004년 10위에서 꾸준히 순위가 상승하고 있고, 알츠하이머병 사망률 역시 통계 작성 이래 10대 사인에 처음 포함됐다.',
  '지난해 알츠하이머병에 의한 사망률은 12.0명으로 전년(9.8명) 대비 22.5% 증가했다.',
  '알츠하이머병 사망률은 10년 전(3.8명)과 비교하면 무려 214.2% 증가했다.',
  '폐렴 

In [150]:
tot_docs = []
tot_docs.extend(paired_docs)
tot_docs.extend(random_docs)

In [152]:
torch.save(tot_docs, 'train_ydocs.pt')
torch.save(n_set_doc, 'train_ndocs.pt')

## VALID

In [6]:
valid_abs = [doc['abstractive'] for doc in valid]
valid_article = [doc['article_original'] for doc in valid]

In [7]:
okt = konlpy.tag.Okt()

In [8]:
word_list = []
for ab in tqdm(valid_abs):
    nouns = okt.nouns(ab)
    word_list.append(' '.join(nouns))

100%|██████████| 10000/10000 [01:34<00:00, 105.95it/s]


In [9]:
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(word_list)

In [10]:
sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [20]:
sim[np.tril_indices(10000)] = 0.0

In [22]:
tot_num = 10000
visited = [0] * tot_num
pairs = []
cnt = 0
for i, r in enumerate(sim):
    sim_max = np.argmax(r)
    if (visited[i] == 0) and (visited[sim_max] == 0) and (0.04 <= r[sim_max] < 0.7):
        visited[i] = visited[sim_max] = 1
        pairs.append((i, sim_max))
        cnt += 2
    if cnt >= 5000:
        break

In [29]:
paired_list = [i for i, v in enumerate(visited) if v == 1]
left_list = [i for i, v in enumerate(visited) if v == 0]

In [38]:
left_list

[27,
 33,
 74,
 100,
 103,
 107,
 110,
 125,
 129,
 133,
 147,
 151,
 172,
 174,
 182,
 183,
 184,
 194,
 206,
 207,
 219,
 224,
 226,
 239,
 243,
 245,
 262,
 290,
 292,
 294,
 297,
 312,
 320,
 325,
 329,
 352,
 357,
 360,
 370,
 382,
 385,
 396,
 404,
 419,
 420,
 424,
 439,
 447,
 467,
 475,
 482,
 486,
 492,
 513,
 514,
 517,
 522,
 546,
 554,
 557,
 583,
 596,
 597,
 598,
 606,
 612,
 616,
 618,
 620,
 630,
 646,
 647,
 649,
 650,
 654,
 662,
 672,
 675,
 681,
 690,
 692,
 694,
 706,
 714,
 715,
 722,
 723,
 725,
 738,
 740,
 747,
 749,
 759,
 760,
 769,
 772,
 774,
 775,
 778,
 780,
 785,
 787,
 788,
 790,
 792,
 795,
 804,
 805,
 807,
 809,
 812,
 818,
 820,
 822,
 825,
 828,
 834,
 838,
 841,
 845,
 847,
 851,
 856,
 860,
 864,
 871,
 872,
 874,
 876,
 882,
 886,
 894,
 897,
 911,
 925,
 929,
 932,
 934,
 940,
 941,
 945,
 947,
 949,
 956,
 959,
 962,
 967,
 968,
 969,
 977,
 987,
 992,
 1009,
 1020,
 1023,
 1025,
 1027,
 1031,
 1032,
 1044,
 1046,
 1049,
 1058,
 1060,
 1063,


In [39]:
paired_docs = [[valid_article[l], valid_article[r]] for l, r in pairs]
n_docs = [valid_article[i] for i in left_list]

In [40]:
n_docs[:3]

[["인천시가 '시민안전이 최상의 복지'를 실현하기 위해 봄철 생활주변에서 발생할 수 있는 안전사고 예방 및 안전의식을 고취시킬 수 있는 다양한 시책을 펼치고 있다.",
  '4일 시에 따르면 세월호 사건 이후 우리사회에 만연한 안전불감증을 개선하고, 시설물에 대한 대대적 안전점검을 통해 안전사회를 실현하기 위해 2015년부터 국가안전대진단을 실시하고 있다.',
  "시는 올해도 지난 2월 18일부터 4월 19일까지 61일간 인천시 및 군·구와 함께 7개 분야 2천개소의 안전시설물을 대상으로 민간전문가, 시민들이 참여하는 '2019 국가안전대진단'을 통해 시민안전을 확인하고 있다.",
  '점검 대상시설은 시, 군·구에서 위험시설로 선정·관리하고 있는 시설로 최근 사고 발생 시설 및 시민 관심 분야, 위험성이 높다고 판단되는 시설 등에 대해서는 시설·소방·전기·가스 등 점검분야별로 효율적인 점검이 이뤄진다.',
  '지난 2일 기준 연인원 3천318명이 참여, 91%의 점검율을 보이고 있으며 오는 19일까지 점검완료를 목표로 한다.',
  '특히 진단결과 보수·보강이 시급한 사항은 즉시 조치하고, 추가 진단이 필요한 사항에 대해서는 정밀안전점검을 실시할 계획이다.',
  '이와 함께 화재안전조사 특별대상에 대해서도 일제 점검을 진행하는 등 대형사고 예방에 만전을 기하고 있다.',
  '시 관계자는 "시민들께서 국가안전대진단에 적극 동참할 수 있도록, 홍보물 배부 및 유튜브 채널 등을 이용한 다양한 홍보를 전개하고 있다"면서, "이번 기간을 통해 생활주변의 위험요소를 다시 한번 살펴 만약의 사고를 미연에 방지하는 기회가 됐으면 한다"고 말했다.'],
 ['지난달 구직급여 지급액이 6803억원으로 전년대비 784억원(13%) 증가했다.',
  '올해 들어 지난달까지 누적 구직급여 지급액은 6조 8900억원으로 지난해 연간 지급액(6조 4523억원)을 넘어섰다.',
  "[자료:고용노동부] 고용노동부가 11일 발표한 '고용행정 통계로 본 10월 노동시장의 주요 특징

In [41]:
torch.save(paired_docs, 'valid_ydocs.pt')
torch.save(n_docs, 'valid_ndocs.pt')